In [39]:
import json
import pprint

with open('cities_usa_Scale8.topo.json') as f:
    d = json.load(f)
    print(d['objects']['citiesScale8']['geometries'][0]["properties"].keys())
#    pprint.pprint(d)

dict_keys(['SCALERANK', 'NATSCALE', 'LABELRANK', 'FEATURECLA', 'NAME', 'NAMEPAR', 'NAMEALT', 'DIFFASCII', 'NAMEASCII', 'ADM0CAP', 'CAPIN', 'WORLDCITY', 'MEGACITY', 'SOV0NAME', 'SOV_A3', 'ADM0NAME', 'ADM0_A3', 'ADM1NAME', 'ISO_A2', 'NOTE', 'LATITUDE', 'LONGITUDE', 'CHANGED', 'NAMEDIFF', 'DIFFNOTE', 'POP_MAX', 'POP_MIN', 'POP_OTHER', 'RANK_MAX', 'RANK_MIN', 'GEONAMEID', 'MEGANAME', 'LS_NAME', 'LS_MATCH', 'CHECKME', 'MAX_POP10', 'MAX_POP20', 'MAX_POP50', 'MAX_POP300', 'MAX_POP310', 'MAX_NATSCA', 'MIN_AREAKM', 'MAX_AREAKM', 'MIN_AREAMI', 'MAX_AREAMI', 'MIN_PERKM', 'MAX_PERKM', 'MIN_PERMI', 'MAX_PERMI', 'MIN_BBXMIN', 'MAX_BBXMIN', 'MIN_BBXMAX', 'MAX_BBXMAX', 'MIN_BBYMIN', 'MAX_BBYMIN', 'MIN_BBYMAX', 'MAX_BBYMAX', 'MEAN_BBXC', 'MEAN_BBYC', 'COMPARE', 'GN_ASCII', 'FEATURE_CL', 'FEATURE_CO', 'ADMIN1_COD', 'GN_POP', 'ELEVATION', 'GTOPO30', 'TIMEZONE', 'GEONAMESNO', 'UN_FID', 'UN_ADM0', 'UN_LAT', 'UN_LONG', 'POP1950', 'POP1955', 'POP1960', 'POP1965', 'POP1970', 'POP1975', 'POP1980', 'POP1985', '

In [38]:
with open('US_cities_States20190708.json', "r") as f:
    cities = json.load(f)
    
    with open('cities_usa_Scale8.topo.json') as g:
        topo = json.load(g)
        print("Number cities:", len(cities))
           
        for i in range(len(topo['objects']['citiesScale8']['geometries'])):
            for city in cities:
                
                # Check that New York is considered. Might be written: New York City
                if city["city_name"] == topo['objects']['citiesScale8']['geometries'][i]["properties"]["NAME"]:

                    topo['objects']['citiesScale8']['geometries'][i]["properties"]["count"] = city["count"]
                    topo['objects']['citiesScale8']['geometries'][i]["properties"]["population"] = city["population"]
            
        print("Write ..")    
        with open('cities_usa_Scale8_CountPopul_20190708.topo.json', 'w', encoding='utf-8') as wr:
            json.dump(topo, wr, ensure_ascii=False, indent=4)

Number cities: 5803
Faribault
Mankato
Albert Lea
Willmar
Brainerd
Crookston
Hardin
Glendive
Dillon
Polson
Devils Lake
Burley
Wallace
Kennewick
Centralia
Glendale
Safford
Casa Grande
Mesa
Lake Havasu City
Berkeley
National City
Mendocino
Paso Robles
Riverside
Delano
San Mateo
Vallejo
Glenwood Springs
Aurora
Greeley
Tonopah
Deming
Truth or Consequences
Las Vegas
Farmington
Springfield
Tillamook
Ontario
La Grande
Richfield
Nephi
Lander
Powell
Paragould
Iowa City
Ottumwa
Spencer
Ft. Dodge
Hutchinson
Kansas City
Lawrence
Garden City
Manhattan
Hays
Goodland
Independence
Kirksville
Kearney
Grand Island
Alliance
Bartlesville
Enid
Ardmore
McAlester
Stillwater
Lead
Slidell
Lake Charles
Metairie
New Iberia
Bryan
San Marcos
Longview
McAllen
Harlingen
Alice
New Braunfels
Cleburne
Brownwood
Alpine
Van Horn
Big Spring
Vernon
Childress
Hereford
Dalhart
Texas City
Pasadena
Baytown
Arlington
New London
Stamford
Waterbury
New Bedford
Springfield
Salem
Pittsfield
Montpelier
Auburn
Florence
Winter Haven
Me